In [2]:
import keras
import numpy as np

path=keras.utils.get_file('nietzche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print("corpus length",len(text))

606208/600901 [==============================] - 2s 4us/step
corpus length 600901


In [6]:
sentences=[]
next_char=[]
maxlen=60
step=3
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_char.append(text[i+maxlen])
char=sorted(list(set(text)))
print("no of characters are",len(char))


no of characters are 59


In [15]:
char_index={x:char.index(x) for x in char}

In [18]:
x=np.zeros((len(sentences),maxlen,len(char)),dtype=np.bool)
y=np.zeros((len(sentences),len(char)),dtype=np.bool)
for i,sentence in enumerate(sentences):
    for j,character in enumerate(sentence):
        x[i,j,char_index[character]]=1
    y[i,char_index[next_char[i]]]=1    

In [26]:
from keras import layers

model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(char))))
model.add(layers.Dense(len(char),activation='softmax'))

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               96256     
_________________________________________________________________
dense_1 (Dense)              (None, 59)                7611      
Total params: 103,867
Trainable params: 103,867
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
model.compile(optimizer='RMSprop',loss='categorical_crossentropy',metrics=['accuracy'])


In [30]:
print(len(sentences))

200281


In [34]:
def sampling(prediction,p=1.0):
    pred=np.asarray(prediction).astype('float64')
    pred=np.log(pred)/p
    pred=np.exp(pred)
    pred=pred/np.sum(pred)
    probab=np.random.multinomial(1,pred,1)
    return np.argmax(probab)

In [ ]:
import random
model.fit(x,y,batch_size=128,epochs=1,verbose=0)
#randomly generate text
start_ind=random.randint(0,len(text)-maxlen-1)
sample_txt=text[start_ind:start_ind+maxlen]
print("The generated text___",sample_txt)
    
for temperature in [.2,.5,1]:
    for i  in range(400):
        sample=np.zeros((1,maxlen,len(char)),dtype=bool)
        for j,charac in enumerate(sample_txt):
            sample[0,round(j),char_index[charac]]=1
        preds=model.predict(sample)[0]
        next_ch=sampling(preds,temperature)
        sample_txt+=char[next_ch]
        sample_txt=preds[1:]
    print("Sample_text: temperature=",temperature,'---')
    print(sample_txt)
        

                